# 일반 Chain에 대화기록만 추가

In [23]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser


# 프롬프트 정의
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 Question-Answering 챗봇입니다. 주어진 질문에 대한 답변을 제공해주세요.",
        ),
        # 대화기록용 key 인 chat_history 는 가급적 변경 없이 사용하세요!
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "#Question:\n{question}"),  # 사용자 입력을 변수로 사용
    ]
)

# llm 생성
llm = ChatOpenAI()

# 일반 Chain 생성
chain = prompt | llm | StrOutputParser()


## 대화를 기록하는 체인 생성(chain_with_history)

In [24]:
# 세션 기록을 저장할 딕셔너리
store = {}


# 세션 ID를 기반으로 세션 기록을 가져오는 함수
def get_session_history(session_id):
    print(f"[대화 세션ID]: {session_id}")
    if session_id not in store:  # 세션 ID가 store에 없는 경우
        # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
        store[session_id] = ChatMessageHistory()
    return store[session_id]  # 해당 세션 ID에 대한 세션 기록 반환


chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,  # 세션 기록을 가져오는 함수
    input_messages_key="question",  # 사용자의 질문이 템플릿 변수에 들어갈 key
    history_messages_key="chat_history",  # 기록 메시지의 키
)


## 첫 번째 질문 실행

In [27]:
chain_with_history.invoke(
    # 질문 입력
    {"question": "나의 이름은 후추야."},
    # 세션 ID기준으로 대화를 기록합니다.
    config = {"configurable":{"session_id": "session_1"}}
)

[대화 세션ID]: session_1


'알겠어요, 후추님. 궁금하신 사항이 있으시면 언제든지 물어보세요. 도와드릴 수 있는 게 있으면 알려주세요. :)'

## 이어서 질문 실행


In [28]:
chain_with_history.invoke(
    # 질문 입력
    {"question": "내이름이 뭐라고?"},
    # 세션 ID기준으로 대화를 기록합니다.
    config = {"configurable":{"session_id": "session_1"}}
)

[대화 세션ID]: session_1


'후추님이라고 알고 있어요. 궁금하신 점이 있거나 도움이 필요하신 게 있나요? 부담 갖지 마시고 물어보세요! :)'

In [32]:
# --- 1) 필요한 컴포넌트 임포트 ------------------------------------------
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
# 왜: ChatPromptTemplate로 시스템/휴먼 메시지 템플릿을 만들고,
#     MessagesPlaceholder로 '과거 대화 기록'이 끼어들 위치를 지정합니다.

from langchain_community.chat_message_histories import ChatMessageHistory
# 왜: 메모리(대화 기록)를 메모리 내에 간단히 저장하는 기본 구현체.
#     프로덕션에서는 Redis/DB 백엔드로 교체 가능.

from langchain_core.chat_history import BaseChatMessageHistory
# 왜: 히스토리 객체의 공통 인터페이스(타입 힌트/확장 시 사용).

from langchain_core.runnables.history import RunnableWithMessageHistory
# 왜: 아무 Runnable(chain)이든 '대화 히스토리 기능'을 덧씌워주는 어댑터.
#     세션별로 히스토리를 꺼내어 프롬프트의 placeholder에 자동 주입해줌.

from langchain_openai import ChatOpenAI
# 왜: OpenAI 챗 모델을 LangChain LLM 인터페이스로 사용.

from langchain_core.output_parsers import StrOutputParser
# 왜: 모델의 응답 메시지(구조체)를 '순수 문자열'로 파싱.


# --- 2) 프롬프트 정의 ----------------------------------------------------
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 Question-Answering 챗봇입니다. 주어진 질문에 대한 답변을 제공해주세요.",
        ),
        # 왜: 이전 턴들의 대화가 들어갈 자리.
        #    반드시 variable_name을 프롬프트와 Runnable 설정에서 동일하게 써야 함.
        MessagesPlaceholder(variable_name="chat_history"),
        # 왜: 사용자의 현재 질문은 {question} 변수로 받을 것임.
        ("human", "#Question:\n{question}"),
    ]
)
# 흐름: 실행 시점에 {question}과 chat_history가 채워져 최종 메시지 리스트가 만들어짐.


# --- 3) LLM 및 체인 구성 -------------------------------------------------
llm = ChatOpenAI()
# 왜: 기본 OpenAI 챗 모델(기본값 모델명은 환경/버전에 따라 다를 수 있음).
#     명시하고 싶으면 ChatOpenAI(model="gpt-4o-mini")처럼 지정.

# 일반 Chain: prompt → llm → 문자열 파서
chain = prompt | llm | StrOutputParser()
# 왜: 파이프 연산자(|)로 Runnable들을 연결.
#     1) 템플릿 렌더 → 2) 모델 호출 → 3) 문자열로 변환.
# 반환: 최종적으로 str를 돌려주는 Runnable이 됨.


# --- 4) 세션별 히스토리 저장소 준비 --------------------------------------
store = {}
# 왜: {session_id: ChatMessageHistory()} 형태의 in-memory 딕셔너리.
#     데모/테스트용. 서버 재시작 시 데이터 사라짐(영속성 없음).


# --- 5) 세션 ID로 히스토리 객체를 꺼내는 팩토리 함수 -----------------------
def get_session_history(session_id):
    print(f"[대화 세션ID]: {session_id}")
    if session_id not in store:
        # 왜: 처음 보는 세션이면 새 히스토리 객체를 생성해 등록.
        store[session_id] = ChatMessageHistory()
    return store[session_id]
# 흐름: RunnableWithMessageHistory가 이 함수를 호출해
#       현재 요청의 세션에 맞는 히스토리를 가져오고, 그 내용을 MessagesPlaceholder에 주입.


# --- 6) 히스토리 어댑터로 감싼 체인 만들기 -------------------------------
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,          # 왜: 세션 히스토리를 어디서/어떻게 꺼낼지 알려줌.
    input_messages_key="question",     # 왜: 프롬프트에서 {question} 변수에 사용자 발화를 넣을 것임을 명시.
    history_messages_key="chat_history",  # 왜: 프롬프트에서 MessagesPlaceholder의 변수명과 일치시켜야 함.
)
# 내부 동작:
# - invoke() 호출 시 config.configurable.session_id를 읽어 get_session_history(session_id) 호출
# - 해당 세션의 지난 메시지들을 chat_history 위치에 삽입
# - 이번 턴의 사용자 질문을 question 변수로 주입
# - 응답을 생성한 후에는 이번 턴의 user/ai 메시지도 히스토리에 자동 반영


# --- 7) 세션 1차 호출 ----------------------------------------------------
chain_with_history.invoke(
    {"question": "나의 이름은 후추야."},                 # 무엇: 현재 턴 사용자 입력
    config={"configurable": {"session_id": "session_1"}}  # 왜: 어떤 세션 히스토리를 쓸지 식별
)
# 흐름:
#  - session_1 히스토리가 없으므로 새로 생성
#  - chat_history는 비어 있음(첫 턴)
#  - 모델이 답변 생성 → 히스토리에 (user, ai) 한 쌍이 저장

# --- 8) 같은 세션으로 2차 호출(대화 지속) --------------------------------
chain_with_history.invoke(
    {"question": "내 이름이 뭐라고?"},
    config={"configurable": {"session_id": "session_1"}}
)
# 흐름:
#  - get_session_history("session_1")로 이전 턴의 (user, ai) 기록을 로드
#  - MessagesPlaceholder(chat_history)에 지난 대화가 들어간 상태로 프롬프트 구성
#  - 모델은 문맥을 보고 "방금 전 너의 이름…" 질문에 맥락 맞는 답을 생성
#  - 이번 턴 (user, ai)도 히스토리에 추가 → 다음 턴에 누적 반영




[대화 세션ID]: session_1
[대화 세션ID]: session_1


'당신의 이름은 후추야입니다. 😊'

In [38]:
store['session_1'].messages  # 세션 1의 누적 대화 기록 확인 가능

[HumanMessage(content='나의 이름은 후추야.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='알겠어요. 반가워요, 후추야! 저는 당신을 돕기 위해 여기 있어요. 궁금한 것이 있으면 언제든지 물어보세요. 😊', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='내 이름이 뭐라고?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='당신의 이름은 후추야입니다. 😊', additional_kwargs={}, response_metadata={})]

In [47]:
chain_with_history.invoke(
    {"question": "파이썬에서 리스트와 튜플의 차이점은 뭐야?"},
    config={"configurable": {"session_id": "session_2"}})    

[대화 세션ID]: session_2


'리스트와 튜플의 주요 차이점은 다음과 같습니다:\n\n1. 리스트(List):\n- [ ] 기호를 사용하여 정의\n- 가변(mutable) 자료형: 요소의 추가, 삭제, 수정이 가능\n- 순서가 있음\n- 인덱스를 사용하여 요소 접근 가능\n- 대괄호([])를 사용하여 요소에 접근\n\n예시:\n```python\nmy_list = [1, 2, 3, 4, 5]\nmy_list.append(6)  # 요소 추가\nmy_list[2] = 33     # 요소 수정\nprint(my_list)\n```\n\n2. 튜플(Tuple):\n- ( ) 기호를 사용하여 정의\n- 불변(immutable) 자료형: 요소의 변경이 불가능\n- 순서가 있음\n- 인덱스를 사용하여 요소 접근 가능\n- 소괄호(())를 사용하여 요소에 접근\n\n예시:\n```python\nmy_tuple = (1, 2, 3, 4, 5)\nprint(my_tuple[2])  # 인덱스로 요소 접근\n```'